## Arnoldi's methods

In [2]:
import LinearAlgebra
const la = LinearAlgebra
MAX_ITER = 100;

In [3]:
A = [4. -1 0 -1 0 0;
     -1 4 -1 0 -1 0;
     0 -1 4. 0 0 -1;
     -1 0 0. 4 -1 0;
     0 -1 0 -1 4 -1;
     0 0 -1 0. -1 4]

b = [0, 5, 0, 6, -2, 6.]

x0 = zeros(size(b));
# n
v = [1., 0, 2, 0, 1, 1];

### Arnoldi Gram-Schmidt
Input: $A\in\mathbb{R}^{n\times n}=[a_1,...,a_n], v\in\mathbb{R}^{n}$ any vector

In [4]:
function arnoldi(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    # Output: (m + 1)xm matrix H and nx1 a nx(m + 1) matrix V
    n = size(A, 1)
    V = zeros((n, m + 1))
    H = zeros((m + 1, m))
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        for i = 1:j
            H[i, j] = la.dot(A*V[:, j], V[:, i])
        end
        w = A*V[:, j] - V[:, 1:j]*H[1:j, j]
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    return H, V
end

arnoldi (generic function with 1 method)

In [5]:
m = 3
H, V = arnoldi(A, v, m);
display(H)
display(V)

4×3 Matrix{Float64}:
 3.14286  1.92195  -2.77556e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556
 0.0      0.0       0.812257

6×4 Matrix{Float64}:
 0.377964   0.168563    0.289595   -0.585252
 0.0       -0.786629    0.191845    0.228787
 0.755929   0.140469   -0.282287    0.128662
 0.0       -0.393314    0.0959226  -0.699892
 0.377964  -0.0280939   0.750937    0.313906
 0.377964  -0.421408   -0.475959    0.0140224

### Arnoldi-Modified Gram-Schmidt

In [6]:
function arnoldi_modified(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    # Output: (m + 1)xm matrix H and nx1 a nx(m + 1) matrix V
    n = size(A, 1)
    
    H = zeros((m + 1, m))
    V = zeros((n, m + 1))
    
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        w = A*V[:, j]
        for i = 1:j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    return H, V
end

arnoldi_modified (generic function with 1 method)

In [7]:
m = 3
H, V = arnoldi_modified(A, v, m)
display(H)
display(V)

4×3 Matrix{Float64}:
 3.14286  1.92195  -2.9976e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556
 0.0      0.0       0.812257

6×4 Matrix{Float64}:
 0.377964   0.168563    0.289595   -0.585252
 0.0       -0.786629    0.191845    0.228787
 0.755929   0.140469   -0.282287    0.128662
 0.0       -0.393314    0.0959226  -0.699892
 0.377964  -0.0280939   0.750937    0.313906
 0.377964  -0.421408   -0.475959    0.0140224

### Householder Arnoldi

In [8]:
function arnoldi_householder(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m < n
    # Output: nx(m+1) matrix H and an nxm orthonormal matrix V
    n = size(A, 1)
    I = 1.0 * Matrix(la.I, n, n)
    V = zeros((n, m + 1))
    H = zeros((n, m + 1))
    Z = zeros((n, m + 1))

    Z[:, 1] = v
    Q = I
    for j = 1:m + 1
        # w calculus
        val_sign = Z[j, j] >= 0.0 ? 1.0 : -1.0
        beta = val_sign * la.norm(Z[j:end, j])
        z = zeros(n)
        z[j] = beta - Z[j, j] + eps()
        for i = j + 1:n
            z[i] = -Z[i, j]
        end
        w = z / la.norm(z)
        # proyector
        P = I - 2 * w * w'
        # h_{j-1}
        H[:, j] = P * Z[:, j]
        # Pj*...*P2*P1
        Q = P * Q
        V[:, j] = Q'[:, j] # P1*P2...*Pj -> j-ésima
        if j <= m
            Z[:, j + 1] = Q * A * V[:, j]
        end
    end
    # delete the column 0 and return a (m+1)xm matrix H
    return H[1:m + 1, 2:m + 1], V
end

arnoldi_householder (generic function with 1 method)

In [9]:
m = 4
H, V = arnoldi_householder(A, v, m)
show(IOContext(stdout, :limit => false), "text/plain", H)
display(V)

5×4 Matrix{Float64}:
  3.14286      -1.92195       0.0           2.22045e-16
 -1.92195       4.6472        1.18932      -2.77556e-16
 -2.77556e-16   1.18932       3.95556      -0.812257
  2.77556e-16  -1.07499e-16  -0.812257      3.67305
 -5.55112e-17  -6.55942e-18   3.09757e-16   0.114091

6×5 Matrix{Float64}:
 0.377964  -0.168563   -0.289595   -0.585252    0.390312
 0.0        0.786629   -0.191845    0.228787    0.540433
 0.755929  -0.140469    0.282287    0.128662    0.2502
 0.0        0.393314   -0.0959226  -0.699892   -0.310248
 0.377964   0.0280939  -0.750937    0.313906   -0.440352
 0.377964   0.421408    0.475959    0.0140224  -0.45036

Se verifica $AV_m = V_{m+1}H$

In [10]:
isapprox(A*V[:, 1:m], V*H)

true

## Arnoldi’s Method for Linear Systems (FOM)

In [11]:
function FOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64)
    r = b - A*x0
    beta = la.norm(r)
    v = r/beta
    H, V = arnoldi_modified(A, v, m)
    # delete last row of H and last column of V
    H = H[1:m, :]
    V = V[:, 1:m]
    return H, V, beta
end

FOM (generic function with 1 method)

In [12]:
m = 4
H, V, beta = FOM(A, b, x0, m)
y = la.inv(H)[:, 1] * beta
x = x0 + V*y

6-element Vector{Float64}:
 0.9999999999999999
 1.9999999999999998
 0.9999999999999999
 2.0
 0.9999999999999999
 2.0

In [13]:
function restarted_FOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1.0e-6)
    println("x[0] = $x0")
    for i=1:MAX_ITER
        H, V, beta = FOM(A, b, x0, m)
        y = la.inv(H)[:, 1] * beta
        x = x0 + V*y
        println("x[$i] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

restarted_FOM (generic function with 2 methods)

In [14]:
m = 4
restarted_FOM(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 2.0, 0.9999999999999999, 2.0]
x[2] = [1.0, 2.0, 1.0, 2.0, 1.0000000000000002, 2.0]


## Incomplete Orthogonalization (IOM)

### Incomplete Arnoldi Process

In [15]:
function incomplete_arnoldi(A:: Matrix, v:: Vector, m:: Int64, k:: Int64)
    n = size(A, 1)
    V = zeros((n, m+1))
    H = zeros((m+1, m))
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        w = A*V[:, j]
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
    end
    return H, V
end

incomplete_arnoldi (generic function with 1 method)

In [16]:
function IOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, k:: Int64)
    r = b - A*x0
    beta = la.norm(r)
    v = r/beta
    H, V = incomplete_arnoldi(A, v, m, k)
    # delete last row of H and last column of V
    # H = H[1:m, :]
    # V = V[:, 1:m]
    return H, V, beta
end

IOM (generic function with 1 method)

In [17]:
m = 5
k = 3
H, V = IOM(A, b, x0, m, k)
display(H)
display(V)

6×5 Matrix{Float64}:
 4.67327  2.21877   3.747e-15   0.0          0.0
 2.21877  3.35573   0.690815   -4.07868e-14  0.0
 0.0      0.690815  4.32887     0.217999     2.94568
 0.0      0.0       0.217999    3.64213      0.0487366
 0.0      0.0       0.0         2.80489e-14  1.76082
 0.0      0.0       0.0         0.0          1.77754

6×6 Matrix{Float64}:
  0.0       -0.493309   -0.238968  -0.446699  -0.308737    0.295458
  0.497519  -0.0612751   0.7893    -0.354578   0.134578   -0.834818
  0.0       -0.493309   -0.238968  -0.446699  -0.308737    0.295458
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391   -0.222677
 -0.199007  -0.702       0.337983   0.59444    0.0949961   0.171221
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391   -0.222677

### DIOM
We need to keep only the $k$ previous $v_i$  in thr Arnoldi process, and we wish to be able to discard the others. However, if we implement the IOM algoritm, we still face the difficulty that when we compute the solution by formula $x_m = x_0 + V_my_m$, we will again need all the vectors $v_i$. One option would be ti recompute them at the end, but this will essentially double the cost of the algorithm. This raise the question of developing a formula whereby the approximate solution can be easily compute from the previous approximation $x_{m-1}$ and a small number of vectors that are being updated at each step. This proceduce is called DIOM. And, of course, $m$ is increasing.

In [18]:
function DIOM(A:: Matrix, b:: Vector, x0:: Vector, m_max:: Int, k:: Int, tol=1.0e-6)
    n = size(A, 1)
    V = zeros(n, m_max + 1)
    H = zeros(m_max + 1, m_max)
    P = zeros(n, m_max)
    
    r0 = b - A*x0
    beta = la.norm(r0)
    zeta = beta
    V[:, 1] = r0/beta

    println("x[0] = $x0")
    for m = 1:m_max
        w = A*V[:, m]
        # IOM block
        for i = max(1, m - k + 1):m
            H[i, m] = la.dot(w, V[:, i])
            w = w - H[i, m]*V[:, i] 
        end
        H[m+1, m] = la.norm(w)
        V[:, m+1] = w/H[m+1, m]
        # Update LU factorization of Hm, LU approach
        L, U = la.lu(H[1:m, 1:m])
        if U[m ,m] < eps()
            break
        end
        zeta = m == 1 ? beta : -L[m, m-1] * zeta    
        i = m - k + 1
        sum_p = i <= 0 ? 0 : P[:, i:m-1] * U[i:m-1, m]  
        P[:, m] = 1/U[m, m] * (V[:, m] .- sum_p)
        x = x0 + zeta*P[:, m]
        println("x[$m] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

DIOM (generic function with 2 methods)

In [19]:
m = 3
k = 2
DIOM(A, b, x0, m, k)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 1.069915254237288, 0.0, 1.2838983050847455, -0.4279661016949152, 1.2838983050847455]
x[2] = [1.0223756706686236, 1.6864518923080625, 1.0223756706686236, 2.0609195678848975, 0.8310997762432935, 2.0609195678848975]
x[3] = [0.9907832820475928, 1.99163513765398, 0.9907832820475928, 2.0053245062998073, 1.0118246060372202, 2.0053245062998073]


## The Generalized Minimum Residual Method (GMRES)

In [20]:
function omega(H:: Matrix, i:: Int64)
    # Input: (m + 1) x m matrix H and column i
    # Output: (m + 1) x (m + 1) matrix O
    m = size(H, 2)
    O = 1.0 * Matrix(la.I, m + 1, m + 1) 
    hyp = sqrt(H[i, i]^2 + H[i + 1, i]^2)
    si = H[i + 1, i]/hyp
    ci = H[i, i]/hyp
    
    O[i, i] = ci
    O[i, i + 1] = si
    O[i + 1, i] = -si
    O[i + 1, i + 1] = ci
    return O 
end

omega (generic function with 1 method)

In [21]:
function givens(H:: Matrix, beta:: Vector)
    # Input: (m + 1)xm matrix H and (m + 1)x1 vector beta
    # Output: m x m upper triangular matrix R and mx1 vector beta
    m = size(H, 2)
    R = copy(H)
    for i = 1:m
        upper_omega = omega(R, i)
        beta = upper_omega * beta
        R = upper_omega * R # makes 0 (i, i+1) element of H
    end
    # delete last row of R and beta; page 176 book 1
    return R[1:m, :], beta[1:m]
end

givens (generic function with 1 method)

In [22]:
function e_vector(m:: Int64, k:: Int64)
    e = zeros(m)
    e[k] = 1.0
    return e
end

e_vector (generic function with 1 method)

In [23]:
function restarted_GMRES(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1.0e-5)
    # Solve Ax = b using the GMRES method
    # Input: n x n matrix A, n x 1 vector b,
    # initial approximation x0, integer m < n,
    # error tolerance tol
    println("x[0] = $x0")
    for i = 1:MAX_ITER
        r = b - A*x0
        # Arnoldi process
        H, V = arnoldi(A, r, m)
        beta = la.norm(r)
        # GMRES process
        # Solve the (m + 1)xm least-squares problem using Givens transformation
        R, g = givens(H, beta*e_vector(m + 1, 1))
        y = la.inv(R)*g
        x = x0 + V[:, 1:m]*y
        println("x[$i] = $x")
        if la.norm(x0 - x) < tol
            print("Number of iterations: $i")
            return nothing
        end
        x0 = copy(x)
    end
end

restarted_GMRES (generic function with 2 methods)

In [24]:
m = 4
restarted_GMRES(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [1.0000000000000002, 1.9999999999999996, 1.0000000000000002, 2.0, 1.0, 2.0]
x[2] = [0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 2.0, 0.9999999999999999, 2.0]
Number of iterations: 2

In [25]:
function QGMRES(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, k:: Int64, tol=1.0e-5)
    # Solve Ax = b using the GMRES method
    # Input: n x n matrix A, n x 1 vector b,
    # initial approximation x0, integer m < n,
    # error tolerance tol
    println("x[0] = $x0")
    for i = 1:MAX_ITER
        r = b - A*x0
        # Arnoldi process
        H, V = incomplete_arnoldi(A, r, m, k)
        beta = la.norm(r)
        # GMRES process
        # Solve the (m + 1)xm least-squares problem using Givens transformation
        R, g = givens(H, beta*e_vector(m + 1, 1))
        y = la.inv(R)*g
        x = x0 + V[:, 1:m]*y
        println("x[$i] = $x")
        if la.norm(x0 - x) < tol
            print("Number of iterations: $i")
            return nothing
        end
        x0 = copy(x)
    end
end

QGMRES (generic function with 2 methods)

In [26]:
m = 3
k = 2
QGMRES(A, b, x0, m, k)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.9905665343243522, 1.9904398530223675, 0.9905665343243522, 2.0051849096588614, 1.0109017962460918, 2.0051849096588614]
x[2] = [0.9999434000739091, 1.9998868001478183, 0.9999434000739091, 1.9998868001478183, 0.999943400073909, 1.9998868001478183]
x[3] = [0.99999946606654, 1.9999994588963876, 0.99999946606654, 2.0000002934655035, 1.0000006170408617, 2.0000002934655035]
x[4] = [0.9999999967964484, 1.9999999935928967, 0.9999999967964484, 1.9999999935928967, 0.9999999967964484, 1.9999999935928967]
Number of iterations: 4

In [27]:
function DQGMRES(A:: Matrix, b:: Vector, x0:: Vector, m_max:: Int64, k::Int64, tol=1.0e-8)
    n = size(A, 1)
    V = zeros(n, 1)
    H = zeros(m_max + 1, m_max)
    P = zeros(n, m_max)

    r0 = b - A*x0
    gamma0 = la.norm(r0)
    V[:, 1] = r0/gamma0
    println("x[0] = $x0")
    for m = 1:m_max
        w = A*V[:, m]
        for i = max(1, m - k + 1):m
            H[i, m] = la.dot(w, V[:, i])
            w = w - H[i, m]*V[:, i] 
        end
        H[m+1, m] = la.norm(w)
        V = hcat(V, w/H[m+1, m])
        
        # Apply Ωi, i = m - k,... ,m - 1 to the m-th column of bar Hm
        for i = max(1, m - k):m - 1
            upper_omega = omega(H, i)
            H[:, m] = upper_omega * H[:, m]
        end
        # Compute the rotation coefficients cm, sm
        hyp = sqrt(H[m, m]^2 + H[m + 1, m]^2)
        sm = H[m + 1, m]/hyp
        cm = H[m, m]/hyp

        # Apply Ωm to bar H_m and g_m, i.e., Compute:
        gamma = -sm*gamma0
        gamma0 = cm*gamma0
        H[m, m] = cm*H[m, m] + sm*H[m + 1, m]

        i = max(1, m - k)
        P[:, m] = (V[:, m] -  P[:, i:m - 1]*H[i:m - 1, m])/H[m, m]
  
        x = x0 + gamma0*P[:, m]
        println("x[$m] = $x")
        if la.norm(x - x0) < tol
            return x
        end
        # updating for next step
        x0 = copy(x)
        gamma0 = gamma
    end
end

DQGMRES (generic function with 2 methods)

In [28]:
m = 3
k = 2
DQGMRES(A, b, x0, m, k)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 0.8731039585645578, 0.0, 1.0477247502774694, -0.3492415834258231, 1.0477247502774694]
x[2] = [0.8766716154479135, 1.5565693183758973, 0.8766716154479135, 1.8997621498855464, 0.6684704699456592, 1.8997621498855464]
x[3] = [0.9266091450709274, 1.902449128381932, 0.9266091450709274, 1.9251104279744615, 0.9287876058819408, 1.9251104279744615]


## The Conjugate Gradient Algorithm 

In [29]:
function lanczos_method(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1e-6)
    # Input: nxn symmetric matrix A 
    n = size(A, 1)
    V = zeros((n, m + 1))
    beta = zeros(m + 1, 1)
    alpha = zeros(m, 1)
    println("x[0] = $x0")
    for i=1:MAX_ITER
        r = b - A*x0
        beta[1] = la.norm(r)
        V[:, 1] =  r/beta[1]
        for j = 1:m
            prod = j == 1 ? 0 : beta[j]*V[:, j - 1]
            w = A*V[:, j] .- prod
            alpha[j] = la.dot(w, V[:, j])
            beta[j + 1] = la.norm(w)
            if beta[j + 1] < eps()
                m = j
                break
            end
            V[:, j+1] = w/beta[j + 1]
        end
        T = la.Tridiagonal(beta[2:m], alpha[:], beta[2:m])
        y = la.inv(T)[:, 1] * beta[1]
        x = x0 + V[:, 1:m]*y
        println("x[$i] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

lanczos_method (generic function with 2 methods)

In [30]:
m = 3
lanczos_method(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.7315640375864656, 0.9184794147188459, 0.7315640375864656, 1.323855555665741, 0.5480080061784134, 1.323855555665741]
x[2] = [0.9162952947096374, 1.655941988496957, 0.9162952947096374, 1.7321302495216035, 0.9031871567881293, 1.7321302495216033]
x[3] = [0.9889186220531084, 1.889484414260903, 0.9889186220531085, 1.931818235814557, 0.9751155864869308, 1.931818235814557]
x[4] = [0.9959439567438422, 1.9793370865896136, 0.9959439567438423, 1.9804450214285771, 0.9990476163774417, 1.9804450214285771]
x[5] = [0.9996288693413997, 1.9925943035288467, 0.9996288693413996, 1.9969042427382877, 0.9972826257130405, 1.996904242738288]
x[6] = [0.9991786129834699, 1.9982075031913884, 0.9991786129834698, 1.9982757183189264, 0.9993537807541217, 1.9982757183189264]
x[7] = [0.999823410181305, 1.9991073018456744, 0.999823410181305, 1.999464589264727, 0.9996374510804579, 1.999464589264727]
x[8] = [0.9999310435605264, 1.9997409105908446, 0.9999310435605264, 1.9997872

In [31]:
function D_lanczos(A:: Matrix, b:: Vector, x0:: Vector, tol=1e-6)
    # Input: nxn symmetric matrix A 
    n = size(A, 1)

    r = b - A*x0
    beta = la.norm(r)
    zeta = beta
    v =  r/beta
    lambda = 0
    p = zeros(n)
    v0 = zeros(n)
    nu = 0
    println("x[0] = $x0")
    for m = 1:MAX_ITER
        w = A*v - beta * v0
        alpha = la.dot(w, v)
        if m > 1
            lambda = beta/nu
            zeta = -lambda * zeta
        end
        nu = alpha - lambda * beta
        p = 1/nu * (v - beta*p)
        x = x0 + zeta * p
        println("x[$m] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        v0 = copy(v)
        x0 = copy(x)
        w = w - alpha * v
        beta = la.norm(w)
        v = w/beta
    end
end

D_lanczos (generic function with 2 methods)

In [32]:
D_lanczos(A, b, x0)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 1.069915254237288, 0.0, 1.2838983050847455, -0.4279661016949152, 1.2838983050847455]
x[2] = [1.022375670668624, 1.686451892308063, 1.022375670668624, 2.0609195678848975, 0.8310997762432939, 2.0609195678848975]
x[3] = [0.990783282047593, 1.9916351376539798, 0.990783282047593, 2.0053245062998073, 1.0118246060372205, 2.0053245062998073]
x[4] = [1.0, 2.0, 1.0, 1.9999999999999998, 1.0000000000000002, 1.9999999999999998]
x[5] = [0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 1.9999999999999996, 1.0, 1.9999999999999996]


In [33]:
function conjugate_gradient(A:: Matrix, b:: Vector, x0:: Vector, tol=1e-6)
    r0 = b - A*x0
    p = copy(r)
    println("x[0] = $x0")
    for j = 0:MAX_ITER
        Apj = A*p
        alpha = la.dot(r0, r0)/la.dot(Apj, pj)
        x = x0 + alpha*p
        println("x[$j] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # upload for next step 
        r = r0 - alpha*Apj
        beta = la.dot(r, r)/la.dot(r0, r0)
        p = r + beta*p
        x0 = copy(x)
        r0 = copy(r)
    end
end

conjugate_gradient (generic function with 2 methods)

In [34]:
D_lanczos(A, b, x0)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 1.069915254237288, 0.0, 1.2838983050847455, -0.4279661016949152, 1.2838983050847455]
x[2] = [1.022375670668624, 1.686451892308063, 1.022375670668624, 2.0609195678848975, 0.8310997762432939, 2.0609195678848975]
x[3] = [0.990783282047593, 1.9916351376539798, 0.990783282047593, 2.0053245062998073, 1.0118246060372205, 2.0053245062998073]
x[4] = [1.0, 2.0, 1.0, 1.9999999999999998, 1.0000000000000002, 1.9999999999999998]
x[5] = [0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 1.9999999999999996, 1.0, 1.9999999999999996]


In [96]:
function three_term_recurrence_variant(A:: Matrix, b:: Vector, x0:: Vector, tol=1e-4)
    r0 = b - A*x0
    r_1 = copy(r0)
    x_1 = zeros(size(x0, 1))
    rho = 1
    lambda0 = 1
    println("x[0] = $x0")
    for j = 1:MAX_ITER
        # Compute
        Arj = A*r0
        r0r0 = la.dot(r0, r0)
        lambda = r0r0/la.dot(Arj, r0)
        if j > 1
            rho = 1/(1 - lambda/lambda0*r0r0/la.dot(r_1, r_1)/rho)
        end
        x = rho*(x0 - lambda*r0) + (1 - rho)*x_1
        println("x[$j] = $x")
        if la.norm(x - x0) < tol
            break
        end
        r = rho*(r0 - lambda*Arj) + (1 - rho)*r_1
        x_1 = copy(x0)
        r_1 = copy(r0)
        x0 = copy(x)
        r0 = copy(r)
    end
end

three_term_recurrence_variant (generic function with 2 methods)

In [97]:
three_term_recurrence_variant(A, b, x0)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, -1.0699152542372883, 0.0, -1.283898305084746, 0.4279661016949153, -1.283898305084746]
x[2] = [-0.7519426297309695, -1.2403611580345604, -0.7519426297309695, -1.5157767579953259, -0.6112619551172401, -1.5157767579953259]
x[3] = [-0.7520007819516021, -1.755508422469578, -0.7520007819516021, -1.8368169858434036, -0.6520353687935404, -1.8368169858434036]
x[4] = [-0.9331203789147096, -1.8220689396166918, -0.9331203789147096, -1.870008370729975, -0.9084938383980552, -1.870008370729975]
x[5] = [-0.9384999100434439, -1.941608580885393, -0.9384999100434439, -1.9575257397553416, -0.9126686788843739, -1.9575257397553416]
x[6] = [-0.983682626914038, -1.954604341604342, -0.983682626914038, -1.9682954154875605, -0.976627154829443, -1.9682954154875605]
x[7] = [-0.984606464344812, -1.9851272074407733, -0.984606464344812, -1.9896166953579142, -0.9782712037378275, -1.9896166953579142]
x[8] = [-0.9958910124420388, -1.988771745782775, -0.9958910124420388,

In [102]:
function conjugate_residual(A:: Matrix, b:: Vector, x0:: Vector, tol=1e-6)
    r = b - A*x0
    p = copy(r)
    println("x[0] = $x0")
    for j = 1:MAX_ITER
        Ap = A*p
        rAr = la.dot(r, A*r)    # old r
        alpha = rAr/la.dot(Ap, Ap)
        x = x0 + alpha*p
        println("x[$j] = $x")
        if la.norm(x - x0) < tol
            break
        end
        r = r - alpha*Ap        # new r
        beta = la.dot(r, A*r)/rAr
        p = r + beta*p
        # update for the next step
        x0 = copy(x)
    end
end

conjugate_residual (generic function with 2 methods)

In [103]:
conjugate_residual(A, b, x0)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 0.8731039585645579, 0.0, 1.0477247502774696, -0.3492415834258232, 1.0477247502774696]
x[2] = [0.9207874560540965, 1.6056336893520193, 0.9207874560540965, 1.9602436074425722, 0.7138153231752253, 1.9602436074425722]
x[3] = [0.9905665343243518, 1.990439853022367, 0.9905665343243518, 2.005184909658861, 1.0109017962460913, 2.005184909658861]
x[4] = [0.9999999999999999, 1.9999999999999996, 0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 1.9999999999999998]
x[5] = [0.9999999999999999, 1.9999999999999996, 0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 1.9999999999999998]


In [129]:
function GCR(A:: Matrix, b:: Vector, x0:: Vector, tol=1e-6)
    r = b - A*x0
    P = copy(r)
    println("x[0] = $x0")
    for j = 1:MAX_ITER
        Ap = A*P[:, j]
        alpha = la.dot(r, Ap)/la.dot(Ap, Ap)
        x = x0 + alpha*P[:, j]
        println("x[$j] = $x")
        if la.norm(x - x0) < tol
            break
        end
        r = r - alpha*Ap
        beta = zeros(j, 1)
        for i = 1:j
            Ap = A*P[:, i]
            beta[i] = -la.dot(A*r, Ap)/la.dot(Ap, Ap)
        end
        
        p = r + P*beta
        P = hcat(P, p)
        # update for the next step
        x0 = copy(x)
    end
end

GCR (generic function with 2 methods)

In [130]:
GCR(A, b, x0)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 0.8731039585645579, 0.0, 1.0477247502774696, -0.3492415834258232, 1.0477247502774696]
x[2] = [0.9207874560540968, 1.6056336893520198, 0.9207874560540968, 1.9602436074425729, 0.7138153231752253, 1.9602436074425729]
x[3] = [0.9905665343243522, 1.9904398530223673, 0.9905665343243522, 2.0051849096588614, 1.0109017962460916, 2.0051849096588614]
x[4] = [1.0, 1.9999999999999998, 1.0, 2.0000000000000004, 1.0, 2.0000000000000004]
x[5] = [1.0, 1.9999999999999998, 1.0, 2.0000000000000004, 1.0, 2.0000000000000004]
